## Calculating Impact Score of Construction

This notebook follows through the steps taken to clean and prepare the data visualized in our project.

In [1]:
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize

#see all columns in notebook
pd.set_option("display.max.columns", None)
np.set_printoptions(suppress=True)

In [2]:
base_url = "https://phl.carto.com/api/v2/sql"
query = """
SELECT * FROM permits
"""

params = {
    "q":query
}

results = requests.get(base_url, params)

data = results.json()

permits = pd.DataFrame(data['rows'])

Then, let us clean the data. Key here is converting the dates to date time, removing all permits that have not been completed, and convertings parcel_id from string to integer. We also want to get rid of all commercial obserbations


In [3]:
#removing observations that are pending or don't have parcel ID
permits_clean = permits.copy()
permits_clean = permits_clean[permits_clean['permitcompleteddate'].notna()]
permits_clean = permits_clean[permits_clean['parcel_id_num'].notna()]

#removing all commercial observations
permits_clean = permits_clean[permits_clean['commercialorresidential'] != "Commercial"]

#removing all expired/not completed observations
wrong_status = ['Expired','EXPIRED','Cancelled', 'ABANDONED', 'REVOKED','Amendment Applicant Revisions','Amendment Application Incomplete',
                'Amendment Ready For Issue','Withdrawn','Expired Denial','Amendment Review', 'Refused']

permits_clean = permits_clean[~permits_clean['status'].isin(wrong_status)]

#converting the date columns to datetime
permits_clean['permitcompleteddate'] = pd.to_datetime(permits_clean['permitcompleteddate'])
permits_clean['permitissuedate'] = pd.to_datetime(permits_clean['permitissuedate'])

#converting parcel id to numeric floats
permits_clean['parcel_id_num'] = pd.to_numeric(permits_clean['parcel_id_num'], errors='coerce')
permits_clean = permits_clean[permits_clean['parcel_id_num'].notna()]
permits_clean['parcel_id_num'] = permits_clean['parcel_id_num'].astype(int)

Next, we are going to extract all the observations that relate to new construction.

In [4]:
# New Construction related words
conswork = [
    'NEWCON', 'New Construction', 'New construction, addition, GFA change', 
    'New Construction (Stand Alone)'
    #, 'SHELL', 'Combined Lot Line Relocation and New Development'
]

# filerting to get only construction permits
construction_permits = permits_clean[
    permits_clean['typeofwork'].isin(conswork)
]

Grouping permits by unique parcel_id to see how many permits were required for each id.

In [5]:
newcons = construction_permits.groupby('parcel_id_num').agg({'permitissuedate': 'min', 'permitcompleteddate': 'max'})
newcons['permit_count'] = construction_permits.groupby('parcel_id_num').size()

Merging back to parcel data.

In [6]:
import geopandas as gpd

geo_subset = permits_clean[['parcel_id_num', 'the_geom']].drop_duplicates('parcel_id_num')
newconssf = newcons.merge(
    geo_subset[['parcel_id_num', 'the_geom']], 
    on='parcel_id_num', 
    how='left'
)

newconssf = gpd.GeoDataFrame(newconssf,  geometry=gpd.GeoSeries.from_wkb(newconssf["the_geom"]), crs=4326)

newconssf.head()

,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,the_geom,geometry
0,-888210498,2021-04-08 14:25:18+00:00,2022-05-24 20:13:58+00:00,1,0101000020E61000000BBD901AA8CF52C0D83EEA872F06...,POINT (-75.24464 40.04833)
1,-888092327,2019-08-28 13:58:00+00:00,2022-09-30 14:59:04+00:00,1,0101000020E610000057BAAF25AECA52C00F36B183D1F9...,POINT (-75.16688 39.95171)
2,-888052165,2018-05-29 12:20:00+00:00,2020-07-02 10:32:19+00:00,1,0101000020E61000004CC9D1C656C952C0C512CD8830F9...,POINT (-75.14592 39.94679)
3,-885933700,2018-09-18 15:19:00+00:00,2021-08-31 15:05:25+00:00,3,0101000020E6100000480C5B830CC952C0CF3C924F99FC...,POINT (-75.14139 39.97343)
4,-885849680,2017-01-10 15:46:00+00:00,2025-03-06 20:28:48+00:00,67,0101000020E6100000FD9C873221CC52C0A75E3A5A5D07...,POINT (-75.18953 40.05754)


Now, the newconssf dataset contains observations of all new constructions in Philadelphia, along with the permtis they asked for as well as the time taken between their first permit and their last permit. Next, lets call in the OPA property assessment dataset and join it to parcel info.


In [7]:
base_url2 = "https://phl.carto.com/api/v2/sql"
query2 = """
SELECT * FROM opa_properties_public
"""

params2 = {
    "q":query2
}

results2 = requests.get(base_url2, params2)

data2 = results2.json()

opa = pd.DataFrame(data2['rows'])

opa.head()

,cartodb_id,the_geom,the_geom_webmercator,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid
0,1,0101000020E6100000F479BF0AB0CA52C005FA8EAE2FFA...,0101000020110F0000963E1FE56FEB5FC12893A60B7489...,2024-06-06T16:14:11Z,None,SWC PARKWAY,54415333,590,RES CONDO 5+ STY MASONRY,1,SINGLE FAMILY,4,None,None,None,0.0,0.0,0.0,3,0.0,0.0,None,0.0,None,A,8,0,18,1600,4,1600-18 ARCH ST,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,1600-18 ARCH ST UNIT 1804,19103-2026,321300,None,0.0,0.0,NaN,1.0,865.0,None,ALLEN ROBERT T,None,888116468,E,B,2025-04-23T04:00:00Z,001N100444,2025-04-16T04:00:00Z,299000.0,None,None,None,PA,13000,ST,None,ARCH,None,282800.0,38500.0,None,0.0,813.0,None,None,1804,None,I,1925,Y,19103,CMX5,1001069978,08,None,619664882
1,2,0101000020E61000001B48E68F4DCB52C00A27E29352F7...,0101000020110F0000FA2A73757BEC5FC14FD734F04786...,2024-06-06T16:09:33Z,C,179' S OF DICKINSON,54415626,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,31,N,None,None,48.0,0.0,0.0,4,0.0,14.0,None,0.0,None,A,36,0,None,1525,3,1525 S GARNET ST,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,1525 S GARNET ST,19146-4627,232100,None,1.0,3.0,NaN,2.0,588.0,None,HOEPFL MICHAEL BRIAN,None,363218600,E,C,2025-04-23T04:00:00Z,011S180154,2025-04-22T04:00:00Z,255000.0,None,None,None,PA,36220,ST,S,GARNET,None,185680.0,46420.0,F,672.0,920.0,A,None,None,None,I,1923,Y,19146,RSA5,1001233687,22,ROW TYPICAL,619664883
2,3,0101000020E61000005763CB6496CB52C075A906264FFD...,0101000020110F00005B1EAE2BF7EC5FC1ECC538F8E98C...,2024-06-06T16:06:48Z,None,"49'1"" N JEFFERSON",54415624,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,137,None,None,None,64.0,0.0,0.0,4,0.0,16.0,None,0.0,None,A,29,0,None,1506,4,1506 N 28TH ST,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,1506 N 28TH ST,19121-3619,192200,None,1.0,3.0,NaN,2.0,562.0,None,HOUSTON DEBRA A,None,292105200,E,C,2025-04-23T04:00:00Z,012N140070,2025-04-17T04:00:00Z,335000.0,None,None,None,PA,88340,ST,N,28TH,None,153800.0,38400.0,F,992.0,1290.0,None,None,None,None,I,1925,Y,19121,RSA5,1001645378,22,ROW TYPICAL,619664884
3,4,0101000020E61000006EAE16BBC4CA52C08E6E8D04C2F6...,0101000020110F000031567D0993EB5FC18011CDD4A785...,2024-06-06T16:05:30Z,C,76' W JUNIPER ST,54415331,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,29,N,None,None,60.0,0.0,0.0,4,0.0,15.0,None,0.0,None,A,39,0,None,1340,4,1340 MOORE ST,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,1340 MOORE ST,19148-1528,369700,None,1.0,3.0,NaN,2.0,1742.0,None,GALLAGHER OWEN,CUSHWA CAROLYN,394559600,E,C+,2025-04-23T04:00:00Z,014S110233,2025-04-18T04:00:00Z,500000.0,None,None,None,PA,56880,ST,None,MOORE,None,295760.0,73940.0,F,900.0,1416.0,A,None,None,None,I,1925,Y,19148,RSA5,1001376311,22,ROW TYPICAL,619664885
4,5,0101000020E61000004FE8831A7EC952C002C49B7CF501...,0101000020110F00009A58533A68E95FC1DFD1379F1192...,2024-06-06T16:07:12Z,D,"264'4""W OLD YORK RD",54415628,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,203,N,None,None,77.0,91840.0,8160.0,4,0.0,16.0,None,0.0,None,A,43,100

Now lets call the property parcels

In [8]:
parcels = gpd.read_file("../data/DOR_Parcel.geojson")

Joining our new construction data with property parcels gives us parcel level information on new construction project, as well as all other information required for tax property assessment.

In [9]:
newcons_parcels = gpd.sjoin(newconssf, parcels, how='right', predicate='intersects').reset_index(drop=True)
newcons_parcels = newcons_parcels[newcons_parcels['parcel_id_num'].notna()].reset_index(drop=True)
newcons_parcels.head()

,index_left,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,the_geom,OBJECTID,RECSUB,BASEREG,MAPREG,PARCEL,RECMAP,STCOD,HOUSE,SUF,UNIT,STEX,STDIR,STNAM,STDESSUF,ELEV_FLAG,TOPELEV,BOTELEV,CONDOFLAG,MATCHFLAG,INACTDATE,ORIG_DATE,STATUS,GEOID,STDES,ADDR_SOURCE,ADDR_STD,COMMENTS,PIN,FRAC,UNIT_TYPE,STEX_FRAC,STEX_SUF,SEPARATED_RIGHTS,MUNIMENT_TYPE,MUNIMENT_ID,DOR_REVIEW,OPA_REVIEW,PWD_REVIEW,Shape__Area,Shape__Length,geometry
0,4002.0,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,0101000020E6100000F56C95DB64C852C0AAC6A8D398FD...,31,,018N210101,018N210101,0101,018N21,75660.0,2086.0,None,None,NaN,E,SUSQUEHANNA,None,0,9999.0,-9999.0,0,1.0,1899/12/30 00:00:00+00,2003/02/24 00:00:00+00,1,None,AVE,2086 E SUSQUEHANNA AVE,2086 E SUSQUEHANNA AVE,None,1.001509e+09,None,None,None,None,None,None,None,None,None,None,112.089844,56.005864,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812..."
1,14796.0,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,0101000020E6100000BAFBF4EFE5C852C090F2930B23FB...,160,,005N060244,005N060244,0244,005N06,61320.0,115.0,None,None,NaN,None,OLIVE,None,0,9999.0,-9999.0,0,2.0,1899/12/30 13:26:14+00,2003/02/27 00:00:00+00,2,None,ST,115 OLIVE ST,115 OLIVE ST,None,NaN,None,None,None,None,None,None,None,None,None,None,76.414062,36.524652,"POLYGON ((-75.13902 39.96205, -75.13905 39.961..."
2,9543.0,385114.0,2019-08-29 17:12:05+00:00,2019-08-29 17:12:05+00:00,1.0,0101000020E61000003F17BE0C6EC952C0836FD40F99FC...,189,,011N050088,011N050088,0088,011N05,87930.0,1415.0,None,None,NaN,N,7TH,None,0,9999.0,-9999.0,0,1.0,1899/12/30 17:44:08+00,2003/05/22 00:00:00+00,1,None,ST,1415 N 7TH ST,1415 N 7TH ST,None,1.001601e+09,None,None,None,None,None,None,None,None,None,None,236.855469,83.936534,"POLYGON ((-75.14718 39.97343, -75.14719 39.973..."
3,7606.0,306585.0,2014-06-23 13:41:52+00:00,2014-06-23 13:41:53+00:00,1.0,0101000020E61000009902E74866CD52C0617857CF36FB...,278,,059N180051,059N180051,0051,059N18,88640.0,602.0,None,None,NaN,N,43RD,None,0,9999.0,-9999.0,0,1.0,1899/12/30 00:00:00+00,2003/02/04 00:00:00+00,1,None,ST,602 N 43RD ST,602 N 43RD ST,None,1.001653e+09,None,None,None,None,None,None,None,None,None,None,412.636719,100.100581,"POLYGON ((-75.20919 39.9626, -75.20954 39.9625..."
4,12339.0,498900.0,2018-01-12 16:01:32+00:00,2020-01-06 15:17:46+00:00,2.0,0101000020E610000012C030E51DCA52C018B320EE3FFC...,309,,008N240160,008N240160,0160,008N24,37040.0,1322.0,None,None,24.0,W,GIRARD,None,0,9999.0,-9999.0,0,1.0,1899/12/30 13:27:37+00,2003/02/27 00:00:00+00,2,None,AVE,1322-24 W GIRARD AVE,1322-24 W GIRARD AVE,None,NaN,None,None,None,None,None,None,None,None,None,None,2226.949219,259.007479,"POLYGON ((-75.15797 39.97112, -75.15798 39.971..."


Lets also rename the index column because this is the only way it works later on.

In [10]:
newcons_parcels = newcons_parcels.rename(columns={'index_left': 'index_left_orig'})

Our data still contains a lot of commercial properties, so we will join them with property parcels and gets rid of them.

In [11]:
opasf = gpd.GeoDataFrame(opa,  geometry=gpd.GeoSeries.from_wkb(opa["the_geom"]), crs=4326).reset_index(drop=True)

newcons_assessment = gpd.sjoin(newcons_parcels, opasf, how = 'left', predicate='intersects').reset_index(drop=True)

newcons_assessment.head()

,index_left_orig,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,the_geom_left,OBJECTID,RECSUB,BASEREG,MAPREG,PARCEL,RECMAP,STCOD,HOUSE,SUF,UNIT,STEX,STDIR,STNAM,STDESSUF,ELEV_FLAG,TOPELEV,BOTELEV,CONDOFLAG,MATCHFLAG,INACTDATE,ORIG_DATE,STATUS,GEOID,STDES,ADDR_SOURCE,ADDR_STD,COMMENTS,PIN,FRAC,UNIT_TYPE,STEX_FRAC,STEX_SUF,SEPARATED_RIGHTS,MUNIMENT_TYPE,MUNIMENT_ID,DOR_REVIEW,OPA_REVIEW,PWD_REVIEW,Shape__Area,Shape__Length,geometry,index_right,cartodb_id,the_geom_right,the_geom_webmercator,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid
0,4002.0,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,0101000020E6100000F56C95DB64C852C0AAC6A8D398FD...,31,,018N210101,018N210101,0101,018N21,75660.0,2086.0,None,None,NaN,E,SUSQUEHANNA,None,0,9999.0,-9999.0,0,1.0,1899/12/30 00:00:00+00,2003/02/24 00:00:00+00,1,None,AVE,2086 E SUSQUEHANNA AVE,2086 E SUSQUEHANNA AVE,None,1.001509e+09,None,None,None,None,None,None,None,None,None,None,112.089844,56.005864,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",68841.0,68842.0,0101000020E61000000FD6C8CE64C852C0B4990DD698FD...,0101000020110F0000AD8A276B8AE75FC1EF665AA43B8D...,2024-06-06T16:04:52Z,A,SWC AMBER ST,54043074,O50,ROW 3 STY MASONRY,1,SINGLE FAMILY,161,Y,None,None,59.0,100000.0,0.0,3,0.0,12.0,A,0.0,None,B,31,100000.0,None,2086,3,2086 E SUSQUEHANNA AVE,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,2086 E SUSQUEHANNA AVE,19125-1542,420700.0,None,2.0,3.0,NaN,1.0,1372.0,None,HALL ERICA C,None,313030300,E,B-,2022-05-18T04:00:00Z,018N210101,2022-05-06T04:00:00Z,522500.0,None,Y,None,PA,75660,AVE,E,SUSQUEHANNA,None,236560.0,84140.0,F,702.0,1626.0,A,None,None,None,I,2013,Y,19125,RSA5,1.001509e+09,22,ROW TYPICAL,619732867.0
1,14796.0,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,0101000020E6100000BAFBF4EFE5C852C090F2930B23FB...,160,,005N060244,005N060244,0244,005N06,61320.0,115.0,None,None,NaN,None,OLIVE,None,0,9999.0,-9999.0,0,2.0,1899/12/30 13:26:14+00,2003/02/27 00:00:00+00,2,None,ST,115 OLIVE ST,115 OLIVE ST,None,NaN,None,None,None,None,None,None,None,None,None,None,76.414062,36.524652,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",140113.0,140114.0,0101000020E61000009FFBF4EFE5C852C0ACF2930B23FB...,0101000020110F0000AC9E16C265E85FC1627993B1818A...,2024-06-06T16:09:42Z,E,100' W OF FRONT ST,53693799,SR,VACANT LAND RESIDE < ACRE,1,SINGLE FAMILY,367,Y,None,None,30.0,603200.0,0.0,1,NaN,33.0,A,1.0,3,3,5,0.0,None,115,1,115 OLIVE ST,None,None,None,PHILADELPHIA PA,115 OLIVE ST,19123,753900.0,None,3.0,3.0,NaN,3.0,416.0,None,MEADE KELLY A,MEADE THOMAS A SR,055167490,E,None,2020-07-27T04:00:00Z,005N060468,2020-07-13T04:00:00Z,680000.0,None,0,None,PA,61320,ST,None,OLIVE,None,0.0,150700.0,F,988.0,2496.0,A,None,None,None,I,2020,None,19123,CMX3,1.001398e+09,25,ROW MODERN,619794556.0
2,9543.0,385114.0,2019-08-29 17:12:05+00:00,2019-08-29 17:12:05+00:00,1.0,0101000020E61000003F17BE0C6EC952C0836FD40F99FC...,189,,011N050088,011N0

In [12]:
multifamily = newcons_assessment[newcons_assessment['category_code_description'].isin(['MULTI FAMILY', 'APARTMENTS > 4 UNITS', 'MIXED USE', 'SINGLE FAMILY'])]

Our dataset is too big so let us drop the columns we don't need.

In [13]:
columns_to_drop = [
    'index_left_orig', 'the_geom_left', 'OBJECTID', 'RECSUB', 'BASEREG', 'MAPREG', 
    'PARCEL', 'RECMAP', 'STCOD', 'HOUSE', 'SUF', 'UNIT', 'STEX', 'STDIR', 'STNAM', 
    'STDESSUF', 'ELEV_FLAG', 'TOPELEV', 'BOTELEV', 'CONDOFLAG', 'MATCHFLAG', 'INACTDATE', 
    'ORIG_DATE', 'STATUS', 'GEOID', 'STDES', 'ADDR_SOURCE', 'ADDR_STD', 'COMMENTS', 'PIN', 
    'FRAC', 'UNIT_TYPE', 'STEX_FRAC', 'STEX_SUF', 'SEPARATED_RIGHTS', 'MUNIMENT_TYPE', 
    'MUNIMENT_ID', 'DOR_REVIEW', 'OPA_REVIEW', 'PWD_REVIEW', 'Shape__Area', 'Shape__Length', 
    'index_right', 'cartodb_id', 'the_geom_right', 'the_geom_webmercator', 'beginning_point', 
    'book_and_page', 'building_code', 'category_code', 'census_tract', 'location', 
    'mailing_address_1', 'mailing_address_2', 'mailing_care_of', 'mailing_city_state', 
    'mailing_street', 'mailing_zip', 'objectid', 'pin',
    'cross_reference', 'date_exterior_condition', 'depth', 'owner_1', 'owner_2',
    'site_type', 'state_code', 'street_code', 'street_designation', 'street_direction', 
    'street_name', 'suffix', 'topography', 'utility', 'unit','central_air','garage_spaces','fuel',
    'general_construction','geographic_ward','house_extension','house_number','other_building','parcel_number',
    'parcel_shape','separate_utilities','sewer','type_heater','unfinished','view_type','year_built_estimate',
    'zip_code','building_code_new','exempt_land','exterior_condition','fireplaces','garage_type',
    'registry_number','quality_grade','number_of_bathrooms', 'number_of_bedrooms', 'number_of_rooms', 'number_stories',
    'basements','interior_condition','off_street_open', 'frontage', 'homestead_exemption'
]

multifamily_trim = multifamily.drop(columns=columns_to_drop)

In [14]:
multifamily_trim.head()

,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,geometry,assessment_date,building_code_description,category_code_description,exempt_building,market_value,market_value_date,recording_date,sale_date,sale_price,taxable_building,taxable_land,total_area,total_livable_area,year_built,zoning,building_code_description_new
0,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,None,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013,RSA5,ROW TYPICAL
1,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,None,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN
2,385114.0,2019-08-29 17:12:05+00:00,2019-08-29 17:12:05+00:00,1.0,"POLYGON ((-75.14718 39.97343, -75.14719 39.973...",2024-06-06T16:05:50Z,ROW 3 STY MASONRY,MULTI FAMILY,0.0,140000.0,None,2019-09-20T04:00:00Z,2019-09-17T04:00:00Z,265000.0,0.0,140000.0,1496.0,4404.0,2023,RSA5,ROW MODERN
3,306585.0,2014-06-23 13:41:52+00:00,2014-06-23 13:41:53+00:00,1.0,"POLYGON ((-75.20919 39.9626, -75.20954 39.9625...",2024-12-11T20:20:25Z,SEMI/DET 2 STY MASONRY,SINGLE FAMILY,82667.0,310000.0,None,2022-12-07T05:00:00Z,2022-11-29T05:00:00Z,398000.0,165333.0,62000.0,2600.0,1800.0,2015,RSA3,TWIN CONVENTIONAL
5,498900.0,2018-01-12 16:01:32+00:00,2020-01-06 15:17:46+00:00,2.0,"POLYGON ((-75.15797 39.97112, -75.15798 39.971...",2024-06-06T16:12:43Z,STR/OFF+APT 3 STY MASONRY,MIXED USE,0.0,766900.0,None,2022-01-10T05:00:00Z,2021-10-27T04:00:00Z,620000.0,613520.0,153380.0,3550.0,4485.0,1920,CMX4,ROW MIXED-COM/RES-BLT AS RES


Now, lets call and join this information with rco data.

In [15]:
rco = gpd.read_file("https://opendata.arcgis.com/datasets/efbff0359c3e43f190e8c35ce9fa71d6_0.geojson")

In [16]:
construction_rco = gpd.sjoin(rco, multifamily_trim, how='right', predicate='intersects').reset_index(drop=True)

In [17]:
drops = ['index_left','OBJECTID','ORGANIZATION_ADDRESS','MEETING_LOCATION_ADDRESS','PREFFERED_CONTACT_METHOD',
         'PRIMARY_NAME', 'PRIMARY_ADDRESS', 'PRIMARY_EMAIL', 'PRIMARY_PHONE',
         'P_PHONE_EXT', 'ALTERNATE_NAME', 'ALTERNATE_ADDRESS', 'ALTERNATE_EMAIL',
         'ALTERNATE_PHONE', 'A_PHONE_EXT', 'EXPIRATIONYEAR', 'EFFECTIVE_DATE',
         'LNI_ID', 'Shape__Area', 'Shape__Length']

construction_rco_trim = construction_rco.drop(columns=drops)

Let us also create another year for construction completion. This year now matches the construction completion used in the property tax assessment dataset, further underscoring the validity of our methods.

In [18]:
construction_rco_trim['cons_complete'] = construction_rco_trim['permitcompleteddate'].dt.year

In [19]:
construction_rco_trim.head()

,ORGANIZATION_NAME,ORG_TYPE,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,geometry,assessment_date,building_code_description,category_code_description,exempt_building,market_value,market_value_date,recording_date,sale_date,sale_price,taxable_building,taxable_land,total_area,total_livable_area,year_built,zoning,building_code_description_new,cons_complete
0,East Kensington Neighbors Association,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,None,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013,RSA5,ROW TYPICAL,2013
1,New Kensington Community Development Corp,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,None,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013,RSA5,ROW TYPICAL,2013
2,Central Delaware Advocacy Group (CDAG),Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,None,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020
3,Northern Liberties Neighbors Association,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,None,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020
4,West Girard Progress,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,None,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020


Now, let us add the district data.

In [20]:
districts = gpd.read_file('https://opendata.arcgis.com/api/v3/datasets/1ba5a5d68f4a4c75806e78b1d9245924_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')
joined = gpd.sjoin(construction_rco_trim, districts, how='left', predicate='intersects').reset_index(drop=True)

In [23]:
no_cols = ['index_right','OBJECTID_1', 'OBJECTID','SHAPE_LENG','Shape__Area', 'Shape__Length','market_value_date']

joined_trim = joined.drop(columns=no_cols)

joined_trim.head()

,ORGANIZATION_NAME,ORG_TYPE,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,geometry,assessment_date,building_code_description,category_code_description,exempt_building,market_value,recording_date,sale_date,sale_price,taxable_building,taxable_land,total_area,total_livable_area,year_built,zoning,building_code_description_new,cons_complete,DISTRICT
0,East Kensington Neighbors Association,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013,RSA5,ROW TYPICAL,2013,7
1,New Kensington Community Development Corp,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013,RSA5,ROW TYPICAL,2013,7
2,Central Delaware Advocacy Group (CDAG),Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020,1
3,Northern Liberties Neighbors Association,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020,1
4,West Girard Progress,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020,CMX3,ROW MODERN,2020,1


Lets also use the cons_complete date based on what is there in the property tax assessment dataset.

In [24]:
joined_trim['year_built'] = pd.to_numeric(joined_trim['year_built'], errors='coerce')
mask = joined_trim['year_built'] > joined_trim['cons_complete']
joined_trim.loc[mask, 'cons_complete'] = joined_trim.loc[mask, 'year_built']

Now, lets add the census data. Census data was calculated in R and exported.

In [25]:
census = gpd.read_file("../data/philly_acs.gpkg")
census = census.to_crs(joined_trim.crs)

In [26]:
yimby_data = gpd.sjoin(joined_trim, census, how='left', predicate='intersects').reset_index(drop=True)

In [27]:
yimby_data.head()

,ORGANIZATION_NAME,ORG_TYPE,parcel_id_num,permitissuedate,permitcompleteddate,permit_count,geometry,assessment_date,building_code_description,category_code_description,exempt_building,market_value,recording_date,sale_date,sale_price,taxable_building,taxable_land,total_area,total_livable_area,year_built,zoning,building_code_description_new,cons_complete,DISTRICT,index_right,GEOID,med_income_2015,med_income_2016,med_income_2017,med_income_2018,med_income_2019,med_income_2020,med_income_2021,med_income_2022,med_income_2023,med_home_value_2015,med_home_value_2016,med_home_value_2017,med_home_value_2018,med_home_value_2019,med_home_value_2020,med_home_value_2021,med_home_value_2022,med_home_value_2023
0,East Kensington Neighbors Association,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013.0,RSA5,ROW TYPICAL,2013,7,892,421010161001,50729.0,65638.0,69439.0,92656.0,93457.0,94309.0,110350.0,121086.0,124779.0,191400.0,196600.0,236500.0,280100.0,309600.0,314400.0,339100.0,347500.0,405200.0
1,New Kensington Community Development Corp,Other,163078.0,2013-05-02 17:52:02+00:00,2013-05-02 17:52:03+00:00,1.0,"POLYGON ((-75.13125 39.9812, -75.13107 39.9812...",2024-06-06T16:04:52Z,ROW 3 STY MASONRY,SINGLE FAMILY,100000.0,420700.0,2022-05-18T04:00:00Z,2022-05-06T04:00:00Z,522500.0,236560.0,84140.0,702.0,1626.0,2013.0,RSA5,ROW TYPICAL,2013,7,892,421010161001,50729.0,65638.0,69439.0,92656.0,93457.0,94309.0,110350.0,121086.0,124779.0,191400.0,196600.0,236500.0,280100.0,309600.0,314400.0,339100.0,347500.0,405200.0
2,Central Delaware Advocacy Group (CDAG),Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020.0,CMX3,ROW MODERN,2020,1,455,421010367003,NaN,NaN,NaN,NaN,NaN,140000.0,121786.0,120429.0,105455.0,NaN,NaN,NaN,NaN,NaN,512700.0,521200.0,599500.0,621200.0
3,Northern Liberties Neighbors Association,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020.0,CMX3,ROW MODERN,2020,1,455,421010367003,NaN,NaN,NaN,NaN,NaN,140000.0,121786.0,120429.0,105455.0,NaN,NaN,NaN,NaN,NaN,512700.0,521200.0,599500.0,621200.0
4,West Girard Progress,Other,1368811.0,2018-09-04 13:11:00+00:00,2020-05-20 19:02:04+00:00,3.0,"POLYGON ((-75.13902 39.96205, -75.13905 39.961...",2024-06-06T16:09:42Z,VACANT LAND RESIDE < ACRE,SINGLE FAMILY,603200.0,753900.0,2020-07-27T04:00:00Z,2020-07-13T04:00:00Z,680000.0,0.0,150700.0,988.0,2496.0,2020.0,CMX3,ROW MODERN,2020,1,455,421010367003,NaN,NaN,NaN,NaN,NaN,140000.0,121786.0,120429.0,105455.0,NaN,NaN,NaN,NaN,NaN,512700.0,521200.0,599500.0,621200.0


In [28]:
inflation_factor = 1.29  # Assuming 29% inflation from 2015 to 2023
yimby_data['real_income_2015'] = yimby_data['med_income_2015'] * inflation_factor
yimby_data['real_home_value_2015'] = yimby_data['med_home_value_2015'] * inflation_factor

yimby_data['income_pct_change'] = (
    yimby_data['med_income_2023'] - yimby_data['real_income_2015']
) / yimby_data['real_income_2015']

yimby_data['home_value_pct_change'] = (
    yimby_data['med_home_value_2023'] - yimby_data['real_home_value_2015']
) / yimby_data['real_home_value_2015']

yimby_data['impact_score'] = (
    yimby_data['income_pct_change'] + yimby_data['home_value_pct_change']
) / 2


In [30]:
yimby_data = yimby_data.drop(columns=['index_right'])

Normalizing impact score to 100.

In [31]:
yimby_data['impact_score'] = (1 + yimby_data['impact_score']) * 100

Now lets bring in the variance data.